Example on how to run the python203 package 

In [12]:
import logging
import pandas as pd
from datetime import datetime
from python203.volumebacktest import VolumeBacktest, Tradesignals
from python203.backtestevaluator import BacktestRating
from python203.Tokentransfers import TokenTransfer
from pybacktestchain.data_module import get_stocks_data
from web3 import Web3

def run_backtest():
    # Configure logging
    logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

    # Initialize VolumeBacktest
    backtest = VolumeBacktest(
        initial_date=datetime(2010, 1, 1),
        final_date=datetime(2018, 1, 1),
        universe=['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'INTC', 'CSCO'],
        initial_cash=1_000_000,
        verbose=True
    )

    # Fetch market data
    logging.info("Fetching market data...")
    data = get_stocks_data(
        backtest.universe,
        backtest.initial_date.strftime('%Y-%m-%d'),
        backtest.final_date.strftime('%Y-%m-%d')
    )

    if data.empty:
        logging.error("No market data available.")
        return

    # Generate signals and execute trades
    logging.info("Generating trading signals and executing trades...")
    data_with_signals = backtest.compute_signals(data)
    results = Tradesignals(broker=backtest.broker).execute_trades(data_with_signals)

    # Extract the Final Portfolio Value, Sharpe Ratio, and Maximum Drawdown
    final_portfolio_value = results.get("Final Portfolio Value")
    sharpe_ratio = results.get("Sharpe Ratio")
    max_drawdown = results.get("Maximum Drawdown")

    if final_portfolio_value is None or sharpe_ratio is None or max_drawdown is None:
        logging.error("Could not compute performance metrics.")
        return

    # Generate rating
    logging.info("Generating backtest rating...")
    metrics = {
        "Sharpe Ratio": sharpe_ratio,
        "Maximum Drawdown": max_drawdown,
        "Final Portfolio Value": final_portfolio_value
    }
    rater = BacktestRating()
    rating = rater.rate_backtest(metrics)

    # Log results
    print(f"Final Portfolio Value: {final_portfolio_value}")
    print(f"Sharpe Ratio: {sharpe_ratio}")
    print(f"Maximum Drawdown: {max_drawdown}")
    print(f"Backtest Rating: {rating}")

    # Transfer Tokens Based on Rating
    ISSUER_PRIVATE_KEY = "3a078e0c831b014a8ff42a99b0e58718eecd856abf4ca040eaada338b9c33077"
    ISSUER_ADDRESS = Web3.to_checksum_address("0x0390cF896B4a7D984017e6C9d3d17b5A6287a874")
    USER_ADDRESS = Web3.to_checksum_address("0xc2142C6007227C9f1F1C6fe6cb7181Ce491F37a7")

    token_transfer = TokenTransfer(private_key=ISSUER_PRIVATE_KEY, sender_address=ISSUER_ADDRESS)

    try:
        token_transfer.transfer_tokens_from_rating(USER_ADDRESS, rating)
        logging.info("Token transfer completed successfully.")
    except Exception as e:
        logging.error(f"Token transfer failed: {e}")
        return

if __name__ == "__main__":
    run_backtest()

INFO:root:Fetching market data...
INFO:root:Generating trading signals and executing trades...
INFO:root:Generating volume-based signals...
c:\Users\ASUS\anaconda3\Lib\site-packages\pybacktestchain\broker.py:115: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.transaction_log = pd.concat([self.transaction_log, transaction], ignore_index=True)
INFO:root:Generating backtest rating...


Final Portfolio Value: 2980149.234334469
Sharpe Ratio: 0.05367417323795205
Maximum Drawdown: -0.200848051592955
Backtest Rating: 7.63


INFO:root:Sender Balance Before: 153.64 tokens
INFO:root:Recipient Balance Before: 49.36 tokens
INFO:root:Transaction sent. TX hash: 0x9986a0212ad37b9502fa028781a2b5c78e37831096e1b0296e969ddf294864c7
INFO:root:Transaction confirmed. Receipt: AttributeDict({'blockHash': HexBytes('0xedf838b0e073d6da1c2776ca905ab3629d784dcda1afd2c5595c3ddb14e8c3d9'), 'blockNumber': 7474704, 'contractAddress': None, 'cumulativeGasUsed': 35110, 'effectiveGasPrice': 50000000000, 'from': '0x0390cF896B4a7D984017e6C9D3d17b5A6287a874', 'gasUsed': 35110, 'logs': [AttributeDict({'address': '0xA0f0a2D53b3476c50F2Cf24307F8a1Cd3c758254', 'blockHash': HexBytes('0xedf838b0e073d6da1c2776ca905ab3629d784dcda1afd2c5595c3ddb14e8c3d9'), 'blockNumber': 7474704, 'data': HexBytes('0x00000000000000000000000000000000000000000000000069e33493143b0000'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef'), HexBytes('0x0000000000000000000000000390cf896b4a7d984017e